In [41]:
import numpy as np
import pandas as pd
from numpy import linalg as LA

In [42]:
def sigmoid(x, derivative=False):

    if x > 100:
        sigm = 1.
    elif x < -100:
        sigm = 0.
    else:
        sigm = 1. / (1. + np.exp(-x))

    if derivative:
        return sigm * (1. - sigm)
    return sigm

In [45]:
def config(data, skill_dim, concept_dim, lambda_s, lambda_t,
           penalty_weight, tol=1e-3, trade_off_example=0.1, max_iter=40, lr=0.005):

    config = {
        'num_users': data['num_users'],
        'num_questions': data['num_quizs'],
        'num_discussions': data['num_disicussions'],
        'num_attempts': data['num_attempts'],
        'num_skills': skill_dim,
        'num_concepts': concept_dim,
        'lambda_s': lambda_s,
        'lambda_t': lambda_t,
        'penalty_weight': penalty_weight,
        'trade_off_example': trade_off_example,
        'lr': lr,
        'tol': tol,
        'max_iter': max_iter,
    }

    train_set = []
    for (stud, ques, obs, att, res) in data['train']:
        train_set.append((int(stud), int(att), int(ques), float(obs), int(res)))
    config['train'] = train_set

    test_set = []
    for (stud, ques, obs, att, res) in data['test']:
        if int(att) < 100:
            test_set.append((int(stud), int(att), int(ques), float(obs), int(res)))
    
    config['num_attempts'] = 19
    config['test'] = test_set

    return config

In [54]:

#-------------------------------------- Hiperparametros -------------------------------------------------

skill_dim = 3
concept_dim = 5
lambda_s = 0
lambda_t = 0
penalty_weight = 0.01
lr = 0.005
max_iter = 40 
trade_off_example = 0.1
tol = 1e-3


#----------------------------------- Lectura de la Data ----------------------------------------------

dataRN = pd.read_csv('Datos.csv')


dataRN = dataRN.sample(frac=1).reset_index(drop = True)

data_train = []
for (i, j, k, l, m) in dataRN.values:
    data_train.append([i, j, k, l, m])

data_test = data_train[:int(len(data_train) * 0.2)]
for x in data_test:
    data_train.remove(x)

data = {}

data['num_users'] = 17
data['num_quizs'] = 12 
data['num_disicussions'] = 8
data['num_attempts'] = 19
data['train'] = data_train
data['test'] = data_test


#-------------------------------- Asignacion del diccionario config ---------------------------------

model_config = config(data, skill_dim, concept_dim, lambda_s, lambda_t,
                          penalty_weight, lr=lr, max_iter=max_iter, trade_off_example=trade_off_example)


#------------------------------------ Inicializacion de matrices y variables-------------------------------

test_set = model_config['test']
train_data = model_config['train']
num_users = model_config['num_users']
num_skills = model_config['num_skills']
num_attempts = model_config['num_attempts']
num_concepts = model_config['num_concepts']
num_questions = model_config['num_questions']
lambda_s = model_config['lambda_s']
lambda_t = model_config['lambda_t']
penalty_weight = model_config['penalty_weight']
lr = model_config['lr']
tol = model_config['tol']
max_iter = model_config['max_iter']

num_examples = model_config['num_discussions']
trade_off_e = model_config['trade_off_example']

binarized_example = True

loss_list = []
val_data = []

S = np.random.random_sample((num_users, num_skills))
T = np.random.random_sample((num_skills, num_attempts,num_concepts))
Q = np.random.random_sample((num_concepts, num_questions)) # Materiales evaluados
E = np.random.random_sample((num_concepts, num_examples))  # Materiales no evaluados

total_test_count = 0
sum_square_error, sum_abs_error = [0.] * 2


#---------------------------------- Entrenamiento -------------------------------------------------

for ciclo in range(0, 30): 
    
    lr = lr

    train_question = [] 
    for student, attempt, question, obs, resource in train_data:
        if resource == 0:
            train_question.append((student, attempt, question, obs, resource))

    np.random.shuffle(train_question)
    
    val_data = train_question[:int(len(train_question) * 0.2)]  # se toma el 20% para data de validacion, esta data cambia
    train_question_size = len(train_question) - len(val_data)   # por cada iteracion
    
    for record in val_data:
        try:
            train_data.remove(record)
        except:
            print(record)
            print(record in val_data)
            print(record in train_data)
            print(sorted(train_data, key=lambda x:x[1]))


    iter = 0
    val_q_rmse_list = [1.]
    converge = False
    min_iters = 35


    while not converge:

        np.random.shuffle(train_data)

        if iter > 0:
            S = best_S
            Q = best_Q
            T = best_T
            E = best_E
    
        for (student, attempt, index, obs, resource) in train_data:
            
            # ----------- Paso de actualizacion de Q
            if resource == 0:
            
                grad_q = np.zeros_like(Q[:, index])
                if obs != None:

                    pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), Q[:, index])
                    grad_q = -2. * (obs - pred) * np.dot(S[student, :], T[:, attempt, :])

                Q[:, index] -= lr * grad_q
                Q[:, index][Q[:, index] < 0.] = 0.

                sum = np.sum(Q[:, index])
                if sum != 0:
                    Q[:, index] /= sum
            
            # ----------- Paso de actualizacion de E
            elif resource == 1:

                grad_e = np.zeros_like(E[:, index])
                if obs != None:
                    pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), E[:, index])

                    if binarized_example:
                        grad_e = -2. * trade_off_e * (obs - pred) * pred * (1. - pred) * np.dot(
                            S[student, :], T[:, attempt, :])
                    else:
                        grad_e = -2. * trade_off_e * (obs - pred) * np.dot(S[student, :],
                                                                            T[:, attempt, :])

                E[:, index] -= lr * grad_e
                E[:, index][E[:, index] < 0.] = 0.

                sum = np.sum(E[:, index])
                if sum != 0:
                    E[:, index] /= sum 

    
            # ----------- Paso de actualizacion de S
            grad_s = np.zeros_like(S[student, :])
            if obs != None:
                if resource == 0:

                    pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), Q[:, index])

                    grad_s = -2. * (obs - pred) * np.dot(T[:, attempt, :], Q[:, index])

                elif resource == 1:
                    
                    pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), E[:, index])

                    if binarized_example:
                        grad_s = -2. * trade_off_e * (obs - pred) * pred * (1. - pred) * np.dot(
                            T[:, attempt, :], E[:, index])
                    else:
                        grad_s = -2. * trade_off_e * (obs - pred) * np.dot(T[:, attempt, :],
                                                                                E[:, index])
            grad_s += 2. * lambda_s * S[student, :]


            S[student, :] -= lr * grad_s

            if lambda_s == 0.:
                S[student, :][S[student, :] < 0.] = 0.
                sum = np.sum(S[student, :])

            if sum != 0:
                S[student, :] /= sum

            # ----------- Paso de actualizacion de T
            grad_t = np.zeros_like(T[:, attempt, :])
            if obs != None:
                
                if resource == 0:
                    pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), Q[:, index])

                    grad_t = -2. * (obs - pred) * np.outer(S[student, :], Q[:, index])

                elif resource == 1:

                    pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), E[:, index])

                    if binarized_example:
                        grad_t = -2. * trade_off_e * (obs - pred) * pred * (1. - pred) * np.outer(
                            S[student, :],
                            E[:, index])
                    else:
                        grad_t = -2. * trade_off_e * (obs - pred) * np.outer(S[student, :],
                                                                                E[:, index])
            grad_t += 2. * lambda_t * T[:, attempt, :]

            if resource == 0:
                if attempt == 0:
                    diff = T[:, attempt + 1, :] - T[:, attempt, :]
                    diff[diff > 0.] = 0.

                    # Penalizacion
                    diff[diff < 0.] = -1.
                    grad_t -= penalty_weight * diff
                        
                elif attempt == num_attempts - 1:
                    gap = T[:, attempt, :] - T[:, attempt - 1, :]
                    gap[gap > 0.] = 0.

                    # Penalizacion
                    gap[gap < 0.] = 1.
                    grad_t -= penalty_weight * gap
                    
                else:
                    diff = T[:, attempt, :] - T[:, attempt - 1, :]
                    diff[diff > 0.] = 0.

                    # Penalizacion
                    diff[diff < 0.] = 1.
                    grad_t -= penalty_weight * diff
                    

                    diff = T[:, attempt + 1, :] - T[:, attempt, :]
                    diff[diff > 0.] = 0.

                    # Penalizacion
                    diff[diff < 0.] = -1.

            elif resource == 1:
                if attempt == 0:
                    diff = T[:, attempt + 1, :] - T[:, attempt, :]
                    diff[diff > 0.] = 0.

                    # Penalizacion
                    diff[diff < 0.] = -1.
                    grad_t -= penalty_weight * diff
                    
                    
                elif attempt == num_attempts - 1:
                    diff = T[:, attempt, :] - T[:, attempt - 1, :]
                    diff[diff > 0.] = 0.

                    # Penalizacion
                    diff[diff < 0.] = 1.
                    grad_t -= penalty_weight * diff

                else:
                    diff = T[:, attempt, :] - T[:, attempt - 1, :]
                    diff[diff > 0.] = 0.

                    # Penalizacion
                    diff[diff < 0.] = 1.
                    grad_t -= penalty_weight * diff

                    diff = T[:, attempt + 1, :] - T[:, attempt, :]
                    diff[diff > 0.] = 0.

                    # Penalizacion
                    diff[diff < 0.] = -1.
                    grad_t -= penalty_weight * diff

            T[:, attempt, :] -= lr * grad_t


        #----------- Calculo de la funcion de costo
        
        #----- L1
        loss, square_loss = 0., 0.
        square_loss_q, square_loss_l, square_loss_e = 0., 0., 0.
        q_count, l_count, e_count = 0., 0., 0.

        for (student, attempt, question, obs, resource) in train_data:
            
            if resource == 0:

                """
                    Se hace le producto matricial entre S y T y luego con Q
                    En la matriz S tomamos la que corresponde al estudiante student
                    En la matriz T tomamos de cada capa el intento attempt
                    En la matriz Q tomamos la columna question de cada concepto
                """
                pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), Q[:, question])

                square_loss_q += (obs - pred) ** 2
            
                q_count += 1

            elif resource == 1:
            
                pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), E[:, question])

                if binarized_example:
                    pred = sigmoid(pred)

                square_loss_e += (obs - pred) ** 2
                e_count += 1

        square_loss = square_loss_q + trade_off_e * square_loss_e

        """
            Ahora calculamos el error de las regularizaciones
            Aplicamos regularizacion a S y T
        """

        reg_S = LA.norm(S) ** 2  # norma 2
        reg_T = LA.norm(T) ** 2  # norma 2

        reg_loss = lambda_s * reg_S + lambda_t * reg_T
        
        loss = square_loss + reg_loss

        q_rmse = np.sqrt(square_loss_q / q_count) if q_count != 0 else 0. 
        e_rmse = np.sqrt(trade_off_e * square_loss_e / e_count) if e_count != 0 else 0.

        # --- Penalizacion dada por L2            
        penalty = 0.
        for (student, attempt, index, obs, resource) in train_data:
            if attempt >= 1:
                gap = T[:, attempt, :] - T[:, attempt - 1, :] 
                gap[gap > 0.] = 0. 
                if resource == 0:   
                    diff = np.dot(np.dot(S[student, :], gap), Q[:, index])
                elif resource == 1:                                       
                    diff = np.dot(np.dot(S[student, :], gap), E[:, index])
            
                penalty -= penalty_weight * diff
                
        loss += penalty

        #------- Termina la funcion de costo

        """
        La data de validacion en cada iteracion es tomada
        de forma aleatoria. Esta data solo contiene materiales del tipo evaluado (resource = 0).
        Ademas utiliza las matrices S, T, Q y E optimizadas
        ya que pasaron por el proceso del descenso de gradiente estocastico.
        Por lo tanto, siempre tenemos distintos errores
        """
        
        val_q_count, val_q_rmse, abs_error = [0.] * 3

        for (student, attempt, question, obs, resource) in val_data: 
            if resource == 0:
                
                val_q_count += 1.
                
                pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), Q[:, question])
     
                val_q_rmse += (obs - pred) ** 2
                abs_error += abs(obs - pred)

        if val_q_count == 0:
            val_q_count, val_q_rmse, abs_error = [0.] * 3
        else:
            val_q_rmse = np.sqrt(val_q_rmse / val_q_count)
            mae = abs_error / val_q_count

        #------------------    

        # Criterios de Parada
        if iter == max_iter:
            loss_list.append(loss)
            converge = True
            count_max_iter += 1

        elif iter >= min_iters and abs(val_q_rmse - val_q_rmse_list[-1]) < tol:
            loss_list.append(loss)
            converge = True
 
        elif iter >= min_iters and val_q_rmse >= np.mean(val_q_rmse_list[-3:]):
            converge = True
         
        elif iter >= min_iters and loss >= np.mean(loss_list[-3:]):
            converge = True

        elif val_q_rmse >= val_q_rmse_list[-1]:
            loss_list.append(loss)
            val_q_rmse_list.append(val_q_rmse)
            iter += 1                           

        else:
            loss_list.append(loss)
            val_q_rmse_list.append(val_q_rmse)
            iter += 1

        best_S = S
        best_T = T
        best_Q = Q
        best_E = E

    for record in val_data:  # Estamos fuera del While. La data que quitamos de entreno la volvemos a colocar
        train_data.append(record)
        
        #--------------- Termina training() -----------------


#-------------------------- testing() --------------------------

    val_q_count, val_q_rmse, abs_error = [0.] * 3

    for (student, attempt, question, obs, resource) in test_set:
        if resource == 0:

            val_q_count += 1.
            
            pred = np.dot(np.dot(S[student, :], T[:, attempt, :]), Q[:, question])
    
            val_q_rmse += (obs - pred) ** 2
            abs_error += abs(obs - pred)

    if val_q_count == 0:
        val_q_count, val_q_rmse, abs_error = [0.] * 3
    else:
        val_q_rmse = np.sqrt(val_q_rmse / val_q_count)
        mae = abs_error / val_q_count
        
    test_count, _rmse, _mae = val_q_count, val_q_rmse, mae

    sum_square_error += (_rmse ** 2) * test_count # esto es solo suma((obs - pred) ** 2)
    sum_abs_error += _mae * test_count
    total_test_count += test_count


rmse = np.sqrt(sum_square_error / total_test_count) # root mean square error datos de testing
mae = sum_abs_error / total_test_count  # mean absolute error datos de test

print(f'\nRMSE: {rmse}')
print(f'\nMAE: {mae}')

print('\nTermino satisfactoriamente....\n')

S = best_S
Q = best_Q
T = best_T
E = best_E


RMSE: 0.2379254225855171

MAE: 0.17730242741580224

Termino satisfactoriamente....

